In [0]:
print ("Myself Yash")


In [0]:
print ("Basic file read")
df = (spark.read
        .format("json")
        .option("inferSchema","true")
        .load("file:/Workspace/Users/yashg6200@gmail.com/Streaming-query-udemy/InvoiceProducer/data/invoices.json"))

display(df)

In [0]:
%python
%run "./Functions/basic_functions"

dbutils.fs.ls("file:/Workspace/Users/yashg6200@gmail.com/Streaming-query-udemy/data_spark_streaming/datasets/text/text_data_1.txt")
#af = read_word_count("file:/Workspace/Users/yashg6200@gmail.com/Streaming-query-udemy/data_spark_streaming/datasets/text/text_data_1.txt")
#display(af)

###Reading The Data CSV and JSON

In [0]:
def read_csv(csv):
    df_csv= (spark.read.format("csv")
                .option("header",True)
                .option("inferSchema",True)
                .load(csv))
    return df_csv

def read_json(json):
    df_json= (spark.read.format("json")
                .option("header",True)
                .option("inferSchema",True)
                .option("multiLine",True)
                .load(json))
    return df_json  

In [0]:
df_csv=read_csv("file:/Workspace/Users/yashg6200@gmail.com/Pyspark_learning/DATA and NOTEBOOK/BigMart Sales.csv")
df_json=read_json("file:/Workspace/Users/yashg6200@gmail.com/Pyspark_learning/DATA and NOTEBOOK/drivers.json")

df_csv.display()
df_json.display()

df_csv.printSchema()

## Playing with DDL and Struct type

In [0]:
def schema_change(new_schema):
    df_csv=(spark.read.format("csv")
                .option("header",True)
                .schema(new_schema)
                .load("file:/Workspace/Users/yashg6200@gmail.com/Pyspark_learning/DATA and NOTEBOOK/BigMart Sales.csv"))
    return df_csv



In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

ddl_type= '''
                Item_Identifier STRING,
                Item_Weight STRING,
                Item_Fat_Content STRING, 
                Item_Visibility DOUBLE,
                Item_Type INT,
                Item_MRP DOUBLE,
                Outlet_Identifier STRING,
                Outlet_Establishment_Year INT,
                Outlet_Size STRING,
                Outlet_Location_Type STRING, 
                Outlet_Type STRING,
                Item_Outlet_Sales DOUBLE 
            '''

ddl_csv=schema_change(ddl_type)
ddl_csv.display()
ddl_csv.printSchema()

                                


In [0]:

struct_schema = StructType([
                                StructField('Item_Identifier',StringType(),True),
                                StructField('Item_Weight',IntegerType(),True),
                                StructField('Item_Fat_Content',StringType(),True),
                                StructField('Item_Visibility',StringType(),True),
                                StructField('Item_MRP',StringType(),True),
                                StructField('Outlet_Identifier',StringType(),True),
                                StructField('Outlet_Establishment_Year',StringType(),True),
                                StructField('Outlet_Size',StringType(),True),
                                StructField('Outlet_Location_Type',StringType(),True),
                                StructField('Outlet_Type',StringType(),True),
                                StructField('Item_Outlet_Sales',StringType(),True)
                            ])
ddl_struct=schema_change(struct_schema)
ddl_struct.display()
ddl_struct.printSchema()
                                

##Transformation 

####SELECT

In [0]:
df_select=read_csv("file:/Workspace/Users/yashg6200@gmail.com/Pyspark_learning/DATA and NOTEBOOK/BigMart Sales.csv").select(col("Item_Identifier"),col("Item_Weight"),col("Item_Fat_Content"))
df_select.printSchema()
df_select.display()


###ALIAS 

In [0]:
df_alias=read_csv("file:/Workspace/Users/yashg6200@gmail.com/Pyspark_learning/DATA and NOTEBOOK/BigMart Sales.csv").select(col("Item_Identifier").alias("Item_id"),col("Item_Weight").alias("Weight"),col("Item_Fat_Content").alias("Fat_Content"))
df_alias.printSchema()
df_alias.display()


###FILTER

####Scenario 1 : Simple Filter

In [0]:
df_csv_data=read_csv("file:/Workspace/Users/yashg6200@gmail.com/Pyspark_learning/DATA and NOTEBOOK/BigMart Sales.csv")
#df_csv_data.display()

df_csv_data.filter(col("Item_Fat_Content")=="Regular").display()



####Scneario 2  : AND Clause

In [0]:
df_csv_data.filter( (col('Item_type')== "Soft Drinks") & (col('Item_Weight') <= 6 )).display()

####Scenario 3 : isin() and isNotNull() or isNull()

In [0]:
df_csv_data.filter((col("Outlet_Location_Type").isin("Tier 1","Tier 2")) |
                    (col("Outlet_Size").isNotNull())).display()

###withColumnRenamed

In [0]:

df_csv_data.withColumnRenamed("Item_Identifier","Item_id").display()

####withColumn

####Constant Value with Scneario 1 to add new Column

In [0]:
df_csv_data.withColumn("NEW",lit("This is a new column")).display()


#### New column with expression

In [0]:
df_csv_data.withColumn("Multiply",expr("Item_Weight * 10")).display()

### Update Existing Column with when().otherwise()

In [0]:
df_csv_data.withColumn("Item_Fat_Content",when(col("Item_Fat_Content")=="Regular","reg").otherwise("Non-reg")).display()

In [0]:
df_csv_data.withColumn("Item_Fat_Content",regexp_replace(col("Item_Fat_Content"),"Regular","Reg")) \
            .withColumn("Item_Fat_Content",regexp_replace(col("Item_Fat_Content"),"Low Fat","LF")).display()

####TYPE CASTING 

In [0]:
df_csv_data.withColumn("Item_Weight",col("Item_Weight").cast(StringType())).display()

####SORT/ORDER BY

####Scenario 1 

In [0]:
df_csv_data.sort(col("Item_Weight").desc()).display()

### Scenario 2 

In [0]:
df_csv_data.sort(col("Item_Visibility").asc()).display()

###Scenario 3 

In [0]:
df_csv_data.sort(col("Item_Visibility").asc(),col("Item_Weight").asc()).display()

df_csv_data.sort(["Item_Visibility","Item_Weight"],ascending = [0,0]).display()

#### LIMIT 

In [0]:
df_csv_data.limit(10).display()

###DROP

In [0]:
df_csv_data.drop("Item_Weight","Item_Identifier").display()

df_csv_data.drop("Item_Weight").display()

###DropDuplicates

###Scenario 1 Drops duplicates from all the columns 

In [0]:
df_csv_data.dropDuplicates().display()

#df_csv_data.dropDuplicates().display()

###Seenario 2 Drop dupes from specific columns

In [0]:
df_csv_data.sort(col("Item_Fat_Content").asc()).display()
df_csv_data.sort(col("Item_Fat_Content").asc()).dropDuplicates(["Item_Fat_Content"]).display()



###DISTINCT 

In [0]:
df_csv_data.distinct().display()


###UNION

In [0]:
data1=[(1,'Sid'),(2,'Ram')]
schema1='id String,Name String'

df1=spark.createDataFrame(data1,schema1)

data2=[(1,'Sid'),(3,'Jay'),(4,'Veeru')]
schema2='id String,Name String'
df2=spark.createDataFrame(data2,schema2)

df2.union(df1).distinct().sort(col("id")).display()
df2.union(df1).display()

In [0]:
data3=[('SId',7),('Jay',6)]
schema3='Name String,id String'

df3=spark.createDataFrame(data3,schema3)

df3.union(df1).display() # -------------- NOTE THIS GIVES INCORRECT DATA
df3.unionByName(df1).display()

##STRING FUNCTION

###INITCAP & LOWER & UPPER

In [0]:
df_string_data=df_csv_data.select("Item_Type","Item_Fat_Content","Item_Weight")

df_string_data.withColumn("INITCAP",initcap(col("Item_Type")))  \
              .withColumn("LOWER CASE",lower(col("Item_Type")))  \
                .withColumn("Upper Case",upper(col("Item_Type"))).drop("Item_Type").display()

##DATE FUNCTIONS 

###CURRENT_DATE()
###DATE_DIFF()
###DATE_ADD()
###DATE_SUB()
###DATE_FORMAT()

In [0]:
df_string_data.withColumn("Current_Date",current_date()) \
        .withColumn("Week After",date_add(current_date(),7)) \
        .withColumn("Week Before",date_sub("Current_Date",7)) \
        .withColumn("Week Before Date Add",date_add("Current_Date",-7)) \
        .drop("Item_Weight","Item_Fat_Content") \
        .withColumn("Difference",date_diff(col("Week After"),col("Week Before"))) \
        .withColumn("Format Change",date_format(col("Current_Date"),'dd/MM/yy')).display()

###HANDLING NULLS

#####DROPPING NULL

In [0]:
df_string_data.dropna('all').display()
df_string_data.dropna('any').display()
df_string_data.dropna(subset=['Item_Weight']).display()

### FILLING NULL

In [0]:
df_csv_data.fillna(0).display()

df_csv_data.fillna("Not Available",subset=["Outlet_Size"]).display()

##SPLIT AND INDEXING FUNCTIONS

In [0]:
df_csv_data.withColumn("Market Name",split(col("Outlet_Type"),' ')[0]) \
            .withColumn("Type",split(col("Outlet_Type"),' ')[1]) \
            .select(col("Outlet_Type"),col("Market Name"),col("Type")) \
           .display()

##EXPLODE

In [0]:
df_exp = df_csv_data.withColumn("Market Name",split(col("Outlet_Type"),' ')) \
            .select(col("Market Name"),col("Outlet_Type")) 
df_exp.withColumn("New Explode",explode("Market Name")).display()


###Array_Contains - TO check if a value is in the array or not returns Boolean
### Array_distinct - To distinct the vlaue in the array column 

In [0]:
df_exp.withColumn("Type1_flag",array_contains(col("Market Name"),"Type1")).display()

df_exp.withColumn("Type1_flag",array_distinct(col("Market Name"))).display()


##GROUP_BY AND AGGREGRATE

In [0]:
df_group=df_csv_data.fillna(0,subset=["Item_Weight"]) \
        .fillna("None",subset=["Outlet_Size"]) \
        .select(col("Outlet_Size"),col("Item_Weight"),col("Item_Type"),col("Item_MRP"))

df_group.groupBy(col("Item_Type"),col("Outlet_Size")).agg(sum(col("Item_Weight")).alias("Sum"),max(col("Item_Weight")).alias("Max Item Weight"),sum(col("Item_MRP")).alias("Sum MRP"),avg(col("Item_MRP")).alias("Avg MRP")).sort([col("Item_Type"),col("Outlet_Size")],ascending=[0,0]).display()

##COLLECT_LIST - To group by values in single column

In [0]:
df_group.select(col("Outlet_Size"),col("Item_Type")).groupBy(col("Outlet_Size")).agg(collect_list("Item_Type")).display()

In [0]:
data_collect=[('1','Computer'),
      ('1','Laptop'),
      ('2','Mobile'),
      ('2','Tablet')]

schema_collect='id string, Item string'

df_collect=spark.createDataFrame(data_collect,schema_collect)

df_collect.groupBy(col("id")).agg(collect_list("Item")).display()

###PIVOT

In [0]:
df_pivot=df_csv_data.select(col("Item_Type"),col("Outlet_Size"),col("Item_MRP"))
df_pivot.groupBy("Item_Type").pivot("Outlet_Size").agg(sum("Item_MRP")).display()

###WHEN OTHERWISE ...

####Scenario 1

In [0]:

df_csv_data.select(col("Item_Type")).withColumn("Type",when(col("Item_type")=="Meat","Non-Veg").otherwise("Veg")).display()

#df_csv_data.select(col("Item_Type")).withColumn("Type",regexp_replace(col("Item_Type"),"Meat","Non-Veg")).display()


####Scenario 2

In [0]:
df_csv_data.select(col("Item_Type"),col("Item_MRP")) \
        .withColumn("Expensive",when((col("Item_Type")=="Meat"),"Non-Veg") \
        .when((col("Item_Type")!="Meat") &(col("Item_MRP")>100),"Veg Expensive").otherwise("Veg In-Expensive")).display()

##JOINS

###INNER JOINS

In [0]:
data1=[("1","Sam","d1"),
       ("2","Tom","d2"),
       ("3","Jerry","d3"),
       ("4","Peter","d1"),
       ("5","Tim","d6")]

schema1='id string,name string, dept string'

set1=spark.createDataFrame(data1,schema1)

data2=[("Accounts","d1"),
       ("IT","d2"),
       ("Hr","d3"),
       ("Clerk","d4")]

schema2='Dept_Name string, dept_id string'

set2=spark.createDataFrame(data2,schema2)

set1.join(set2,set1["dept"]==set2["dept_id"]).select(col("name"),col("dept"),col("Dept_Name")).display()



###LEFT, RIGHT, OUTER AND ANTI JOIN

In [0]:
set1.join(set2,set1["dept"]==set2["dept_id"],"left").display()  # LEFT JOIN
# set1.join(set2,set1.dept==set2.dept_id,"left").display()  # LEFT JOIN ANOTHER WAY FOR KEYS 


set1.join(set2,set1["dept"]==set2["dept_id"],"right").display() # RIGHT JOIN


set1.join(set2,set1["dept"]==set2["dept_id"],"outer").display() # OUTER JOIN


set2.join(set1,set1["dept"]==set2["dept_id"],"anti").display() # ANTI JOIN



##WINDOW FUNCTIONS 

###ROW_NUMBER

In [0]:
from pyspark.sql.window import Window

df_csv_data.withColumn("ROW_ID",row_number().over(Window.orderBy("Item_Identifier"))).display()

df_csv_data.withColumn("ROW_ID",row_number().over(Window.partitionBy("Item_Identifier").orderBy("Item_Identifier"))).display()


###RANK AND DENSE_RANK

In [0]:
from pyspark.sql.window import Window

df_csv_data.select(col("Item_Identifier")) \
    .withColumn("Rank",rank().over(Window.orderBy("Item_Identifier"))) \
    .withColumn("DENSE_RANK",dense_rank().over(Window.orderBy("Item_Identifier"))) \
    .withColumn("ROW_NUM",row_number().over(Window.orderBy("Item_Identifier"))) \
    .withColumn("Row_number_Part",row_number().over(Window.partitionBy("Item_identifier").orderBy("Item_identifier"))) \
    .display()

###CUMULATIVE SUM

In [0]:
df_csv_data.select(col("Item_Type"),col("Item_MRP")) \
    .withColumn("CUM_SUM",sum("Item_MRP").over(Window.orderBy("Item_Type").rowsBetween(Window.unboundedPreceding,Window.currentRow))) \
    .withColumn("Cum_sum_part",sum("Item_MRP").over(Window.orderBy("Item_Type").partitionBy("Item_Type").rowsBetween(Window.unboundedPreceding,Window.currentRow))) \
    .display()

## USER DEFINED FUNCTIONS (UDF)

###STEP-1

In [0]:
def my_func(x):
  return x*x

###STEP-2 

In [0]:
my_udf=udf(my_func)

In [0]:
df_csv_data.withColumn("Square",my_udf(col("Item_MRP"))).display()

#DATA_WRITING

##CSV with Writing Modes

In [0]:
data1=[("1","Sam","d1"),
       ("2","Tom","d2"),
       ("3","Jerry","d3"),
       ("4","Peter","d1"),
       ("5","Tim","d6")]

schema1='id string,name string, dept string'

data_csv=spark.createDataFrame(data1,schema1)


In [0]:
dbutils.fs.rm('/FileStore/tables/data.csv',recurse=True)

In [0]:
data_csv.write.format('csv')\
  .option("header",True)\
  .mode("error")\
  .save('/FileStore/tables/data.csv')
# .option('path','/FileStore/tables/data.csv').save() THis is also the option to save the file. 

spark.read.format('csv')\
.option("header",True)\
.option("inferSchema",True)\
.load('/FileStore/tables/data.csv').display()

###Overwrite Method

In [0]:
data_csv.write.format('csv')\
  .option("header",True)\
  .mode("overwrite")\
  .save('FileStore/tables/data.csv')

spark.read.format('csv')\
.option("header",True)\
.option("inferSchema",True)\
.load('/FileStore/tables/data.csv').display()


###APPEND MODE

In [0]:
data_csv.write.format('csv')\
  .option("header",True)\
  .mode("append")\
  .save('FileStore/tables/data.csv')

spark.read.format('csv')\
.option("header",True)\
.option("inferSchema",True)\
.load('/FileStore/tables/data.csv').display()


###IGNORE MODE

In [0]:
data_csv.write.format('csv')\
  .option("header",True)\
  .mode("ignore")\
  .save('FileStore/tables/data.csv')

spark.read.format('csv')\
.option("header",True)\
.option("inferSchema",True)\
.load('/FileStore/tables/data.csv').display()


###PARQUET FILE FORMAT

In [0]:
data_csv.write.format('parquet')\
  .option("header",True)\
  .mode("overwrite")\
  .save('FileStore/tables/data.csv')

spark.read.format('parquet')\
.option("header",True)\
.option("inferSchema",True)\
.load('/FileStore/tables/data.csv').display()

###DELTA FILE FORMAT -Delta table

In [0]:
data_csv.write.format('delta')\
  .option("header",True)\
  .mode("overwrite")\
  .save('/FileStore/tables/data')

spark.read.format('delta')\
.option("header",True)\
.option("inferSchema",True)\
.load('/FileStore/tables/data').display()

### SAVE TABLE
####This command will save the table in Warehouse, since the format is not Delta it wont support Delta logs

In [0]:
data_csv.write.format('parquet')\
  .option("header",True)\
  .saveAsTable('Table1')

##SPARK SQL

###Create View 

In [0]:
data_csv.createOrReplaceTempView("Testing")

spark.sql("Select * from Testing").display()

In [0]:
%sql
Select * from Testing